In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Libraries

In [ ]:
# !pip install protobuf==3.20 --quiet
# import os
# os.kill(os.getpid(), 9)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0 ,MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam


## Load Dataset

In [ ]:
base_path = "/kaggle/input/garbage-classification/garbage_classification"  # original data
output_path = "/kaggle/working/garbage_split"      # new split data
classes = os.listdir(base_path)

print("Classes:", classes)

## Splitting into Train Test validation 

In [ ]:
#create train test validation split
splits = ['train', 'val', 'test']

for split in splits:
    for cls in classes:
        path = os.path.join(output_path, split, cls)
        os.makedirs(path, exist_ok=True)


In [ ]:
# split each class with 70% Train, 20% Val, 10% Test
for cls in classes:
    class_path = os.path.join(base_path, cls)
    images = os.listdir(class_path)

    # Train 70%, Temporary 30%
    train_imgs, temp_imgs = train_test_split(images, test_size=0.30, random_state=42)

    # Val 20%, Test 10%
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.33, random_state=42)

    # -----------------------
    # Copy Train
    # -----------------------
    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img),
                    os.path.join(output_path, "train", cls, img))

    # -----------------------
    # Copy Validation
    # -----------------------
    for img in val_imgs:
        shutil.copy(os.path.join(class_path, img),
                    os.path.join(output_path, "val", cls, img))

    # -----------------------
    # Copy Test
    # -----------------------
    for img in test_imgs:
        shutil.copy(os.path.join(class_path, img),
                    os.path.join(output_path, "test", cls, img))

print("Data splitting completed!")


## Create ImageDataGenerator

In [ ]:
print(output_path)

In [ ]:
train_dir = "/kaggle/working/garbage_split/train"
val_dir   = "/kaggle/working/garbage_split/val"
test_dir  = "/kaggle/working/garbage_split/test"

In [ ]:
image_size = 224
batch_size = 32

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

val_datagen  = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator= train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical')

val_generator= val_datagen.flow_from_directory(
    val_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical'    
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


In [ ]:
print(train_generator.class_indices)

## Build MobileNetV2 Model

In [ ]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(image_size,image_size,3)
)

base_model.trainable = False  # freeze base initially

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(12, activation='softmax')(x)

model = Model(base_model.input, outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Train model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.3, patience=2)]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks = callbacks
)


## Evaluate

In [ ]:
#evaluate
val_loss, val_acc = model.evaluate(val_generator)
print("Validation Accuracy:", val_acc)


## Visualize Accuracy and Loss

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], 'b', label='Train acc')
plt.plot(history.history['val_accuracy'], 'r', label='Val acc')
plt.legend(); plt.title("Accuracy")

plt.subplot(1,2,2)
plt.plot(history.history['loss'], 'b', label='Train loss')
plt.plot(history.history['val_loss'], 'r', label='Val loss')
plt.legend(); plt.title("Loss")
plt.show()

## predict on Test image

In [ ]:
pred = model.predict(test_generator)
labels = pred.argmax(axis=1)     #predicted class(label) index
actual = test_generator.classes #actual class index

## Save model

In [ ]:
model.save("mobilenetv2_garbage_classification.h5")


## Image Prediction Code

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array


# 1. Load Your Saved Model

model = tf.keras.models.load_model("mobilenetv2_garbage_classification.h5")

# List of your 12 class names (replace with your actual folder names)
class_names = ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes',
               'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']




# 2. Load and Preprocess Image

img_path = "/kaggle/input/food-waste/food.jpg"   # <-- replace with your test image

IMG_SIZE = (224, 224)         # Same size you used in training

img = load_img(img_path, target_size=IMG_SIZE)
img_array = img_to_array(img) / 255.0         # normalize
img_array = np.expand_dims(img_array, axis=0) # add batch dim


# 3. Predict

prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction)         # best class index
predicted_class_name = class_names[predicted_class_index]


# 4. Results

print("Raw Prediction:", prediction)
print("Predicted Class Index:", predicted_class_index)
print("Predicted Label:", predicted_class_name)